<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>

# 模型训练准备工作 #
我们注意到，视频 AI 应用的 TrafficCamNet 模型存在一些问题。该模型可能并未完全针对我们的停车场用例进行训练。在实验的余下部分，我们将使用 TAO 工具包微调模型，使其适应我们的环境。模型开发的一般工作流程如下。首先，准备预训练模型和数据。接着，准备配置文件，并开始使用新数据训练模型并评估其性能。如果模型性能令人满意，我们即将其导出。请注意，这一流程不包括推理优化步骤，这些步骤对部署于边缘设备上的视频 AI 应用非常重要。
<p><img src='images/pre-trained_model_workflow.png' width=1080></p>

## 学习目标 ##
在此 Notebook 中，您将学习如何使用 TAO 工具包准备训练视频 AI 模型，学习内容包括：
* 了解模型规范
* 准备数据以供 TAO 工具包使用
* 编辑面向 TAO 工具包任务的规格文件

**目录**<br>
本 Notebook 涵盖以下部分：
1. [Detectnet_v2 物体检测模型](#s1)
    * [目录结构](#s1.1)
    * [模型目标](#s1.2)
2. [准备预训练模型](#s2)
    * [练习 #1 - 查看模型卡](#e1)
3. [准备数据集](#s3)
    * [标记数据](#s3.1)
    * [探索性数据分析](#s3.2)
    * [将视频文件转换为帧图像](#s3.3)
    * [生成标签](#s3.4)
    * [转换为 TFRecord 文件](#s3.5)
    * [练习 #2 数据集转换](#e2)

<a name='s1'></a>
## Detectnet_v2 物体检测模型 ##
如前所述，[TrafficCamNet](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tao/models/trafficcamnet) 专用模型依托于 NVIDIA DetectNet_v2 检测器构建而成，并使用 ResNet18 作为特征提取器。因此，我们使用支持以下子任务的 `detectnet_v2` 任务：
* `dataset_convert`
* `train`
* `evaluate`
* `inference`
* `prune`
* `calibration_tensorfile`
* `export`

<p><img src='images/rewind.png' width=720><p>
    
我们可在命令行中使用 `detectnet_v2 <subtask> <args_per_subtask>` 来调用这些子任务。此外，还可通过使用 `detector_v2 <subtask> --help` 详细了解这些子任务。

<a name='s1.1'></a>
### 目录结构 ###
我们将在项目中使用以下结构，其中 `tao_project` 目录将保留与模型训练和输出相关的大部分素材。

<p><img src='images/project_structure.png' width=740></p>

* 当前目录为 `/dli/task`。使用路径时，更为可靠的方式是使用以 `/dli/task` 为开头的绝对路径，这是因为如不这样做，某些函数将尝试引用其被调用位置的相对路径。
* 更高级别的 `data` 目录表示原始视频数据，而较低级别的 `tao_project/data` 目录代表将用于模型训练的预处理数据。
* 更高级别的 `images` 目录包含本课程中使用的图形，并与视频 AI 模型无关联。
* `spec_files` 目录包含用于 TAO 工具包模型训练的规格文件以及 DeepStream `Gst-nvinfer` 插件配置文件。
* 在我们努力确定经优化的最终产品过程中，`tao_project/models` 目录将保留模型的不同版本。每个文件夹都将保存相应的模型文件（例如 `.tlt` 或 `.etlt`）以及 `labels.txt` 等随附素材。

执行以下单元，为 TAO 工具包设置和创建目录。

In [ ]:
# DO NOT CHANGE THIS CELL
# Set and create directories for the TAO Toolkit experiment
import os

os.environ['PROJECT_DIR']='/dli/task/tao_project'
os.environ['SOURCE_DATA_DIR']='/dli/task/data'
os.environ['DATA_DIR']='/dli/task/tao_project/data'
os.environ['MODELS_DIR']='/dli/task/tao_project/models'
os.environ['SPEC_FILES_DIR']='/dli/task/spec_files'

!mkdir $PROJECT_DIR
!mkdir $DATA_DIR
!mkdir $MODELS_DIR

<a name='s1.2'></a>
### 模型的训练目标 ###
对于视频 AI 应用，我们希望训练一个以 TrafficCamNet 为起点的模型，并为其提供额外的（已标记）数据，以便其适应我们特定的摄像头角度、照明条件和其他环境条件。我们会将**未剪枝的预训练 TrafficCamNet 专用模型**用作起点，并训练一个符合我们用例的自定义**单类物体检测模型**。

<a name='s2'></a>
## 准备预训练模型和数据集 ##
开发者通常首先会从 [NGC](https://ngc.nvidia.com/) 中选择并下载预训练模型，其中既有高度精确的专用模型，也有符合其所选预训练权重的架构。由于经常需要在训练时间、准确性和推理性能之间进行权衡，因此开发者很难立即确定何种模型/架构最适合某个特定用例。在选择最佳候选模型之前，通常要比较多个模型。

以下是一些有助于选择恰当模型的提示：
* 查看模型输入/输出，以确定模型是否适合您的用例。
* 输入格式也是需考虑的一项重要因素。例如，TrafficCamNet 和其他 DetectNet_v2 模型预计输入为 0~1的取值，且输入通道按 RGB 顺序归一化。按 BGR 顺序的模型需要输入预处理/均值消减，而这可能会导致性能欠佳。

我们可以使用 `ngc registry model list <model_glob_string>` 命令获取位于 NGC 模型注册表中的模型列表。例如，我们可以使用 `ngc registry model list nvidia/tao/*` 列出所有可用模型。`--column` 可用于识别兴趣列。NGC 注册 CLI 相关详情请参阅[用户指南](https://docs.nvidia.com/dgx/pdf/ngc-registry-cli-user-guide.pdf)。每个模型下都存在一个剪枝版本和未剪枝版本，前者可支持直接部署，后者可支持针对特定用例使用更多数据重新训练。如要开展可训练，可选用未剪枝版本。`ngc registry model download-version <org>/[<team>/]<model-name:version>` 命令可用于下载注册表中的模型，并可通过 `--dest` 指定下载目录的路径。

<a name='e1'></a>
#### 练习 #1 - 查看模型卡 ####
下载预训练模型。

**说明**：<br>
* 查看 [TrafficCamNet](https://catalog.ngc.nvidia.com/orgs/nvidia/team/tao/models/trafficcamnet) 或 [DetectNet_v2](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/tao/models/pretrained_detectnet_v2) 模型的模型卡，了解在重要模型规格的所在位置。
* 执行以下单元，下载 NGC CLI。
* 执行以下单元，列出所有可用模型。
* 执行以下单元，下载 TrafficCamNet 模型。
* 执行以下单元，检查是否已下载该模型。
* 如果 `labels.txt` 不存在，执行以下单元完成创建。

In [ ]:
# DO NOT CHANGE THIS CELL
# Download the NGC CLI
%env CLI=ngccli_cat_linux.zip
!mkdir -p ngc_assets/ngccli
!wget "https://ngc.nvidia.com/downloads/$CLI" -P ngc_assets/ngccli
!unzip -u "ngc_assets/ngccli/$CLI" \
       -d ngc_assets/ngccli/
!rm ngc_assets/ngccli/*.zip 
os.environ["PATH"]="{}/ngccli/ngc-cli:{}".format("ngc_assets", os.getenv("PATH", ""))

In [ ]:
# DO NOT CHANGE THIS CELL
# List all available models
!ngc registry model list nvidia/tao/* --column name --column repository --column application

In [ ]:
# DO NOT CHANGE THIS CELL
# Download the unpruned pre-trained model from NGC
!ngc registry model download-version nvidia/tao/trafficcamnet:unpruned_v1.0 \
    --dest $MODELS_DIR

In [ ]:
# DO NOT CHANGE THIS CELL
# Download the pruned pre-trained model from NGC
!ngc registry model download-version nvidia/tao/trafficcamnet:pruned_v1.0 \
    --dest $MODELS_DIR

In [ ]:
# DO NOT CHANGE THIS CELL
# Check if models have been downloaded into directory
!ls -rlt $MODELS_DIR

**观察**：<br>
以下是一些需要注意的字段：

<p><img src='images/model_card_tao.png' width=1080></p>

<p><img src='images/encryption_key.png' width=540></p>

<p><img src='images/important.png' width=720></p>

_请注意：我们使用专用 TrafficCamNet 模型作为场景改编的起点。如果课程结束时还有多余时间，您可以随意尝试使用其他模型架构进行实验。在使用 NGC 的专用模型时，加载模型需要有正确的**加密密钥**。用户通过通用模型开展训练时，将能够定义自己的导出加密密钥。此操作可保护专有 IP，并用于解密 DeepStream 应用中的 `.etlt` 模型。_

<a name='s3'></a>
## 准备数据集 ##
我们将使用在 NVIDIA 总部停车场通过同一摄像头拍摄的带标记视频数据来训练模型。须明确的一点是，我们提供的数据有限且不足以从头开始训练模型。通过利用 TAO 工具包和迁移学习，我们可以使用 TrafficCamNet 模型作为起点并训练自定义模型。这是我们在利用 TAO 工具包的场景/域适应功能时的常规操作。

TAO 工具包要求使用特定格式的数据开展训练和评估：
* TAO 工具包中的物体检测任务需要 `KITTI format` 中的数据。
    * `images` 目录包含要训练的图像。
    * `labels` 目录包含对应图像的标签。
    * `kitti_seq_to_map.json` 文件是*可选项*，包含为图像目录中的帧提供的从序列到帧 ID 的映射。如果需要按序列将数据拆分为不同的折叠，此文件很实用。

<p><img src='images/detection_input.png' width=720></p>

* 相比之下，分类任务需要具有以下结构的图像目录，其中每个类都有自己的目录和类名称。

<p><img src='images/classification_input.png' width=720></p>

_您可以参阅 [TAO 工具包用户指南](https://docs.nvidia.com/metropolis/TLT/tlt-user-guide/text/data_annotation_format.html#object-detection-kitti-format)_ ，详细了解数据标记格式。

<a name='s3.1'></a>
### 标记数据 ###
先预览视频，然后再继续。

执行下方单元，观看视频。

In [ ]:
# DO NOT CHANGE THIS CELL
# View the video
from IPython.display import Video

Video("data/126_206-A0-3_raw.mp4", width=720)

除视频源之外，我们还需添加标签来评估推理结果（将实际物体与深度学习模型检测到的物体进行比较），并扩展用于训练的真值。一般来说，整个过程很耗时，但迁移学习会大幅改善这一情况。公开可用的标记工具有很多。我们的数据集标记数据由我们使用 [Vatic](https://github.com/cvondrick/vatic) 手动生成，并以 JSON 格式提供。每个条目均以 `track_id` 为开头，代表记录中每辆车的唯一索引。`track_id` 包含一组边界框及其各自的边框位置。下方列出了标记数据格式的元素：

<p><img src="images/vatic.jpg" width=720></p>

这是视频中 JSON 文件的快照。我们主要关注为物体检测模型捕获的边界框坐标：
* **xbr**：[0, 帧宽] 之间的整数，表示相对于帧大小，边界框在坐标中的最右侧位置。<br />
* **xtl**：[0, 帧宽] 之间的整数，表示相对于帧大小，边界框在坐标中的最左侧位置。<br />
* **ybr**：[0, 帧高] 之间的整数，表示相对于帧大小，边界框在坐标中的最底部位置。<br />
* **ytl**：[0, 帧高] 之间的整数，表示相对于帧大小，边界框在坐标中的最顶部位置。<br />
<p><img src="images/json_structure.png" width=720></p>

执行以下单元，以 JSON 格式预览标记数据。

In [ ]:
# DO NOT CHANGE THIS CELL
# Preview the annotation
!cat $SOURCE_DATA_DIR/126_206-A0-3_json_sample.txt

<a name='s3.2'></a>
### 探索性数据分析 ###
我们可以使用 [Pandas DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) 分析和预处理数据。为此，我们继续将 JSON 文件转换为 .csv 文本文件，否则整个过程会非常耗时。

执行以下单元，分析标记数据文件中包含的数据。

In [ ]:
# DO NOT CHANGE THIS CELL
# Load the .csv into a DataFrame
import ast
import pandas as pd

annotated_frames=pd.read_csv('data/annotation.csv', converters={2:ast.literal_eval})
print("Length of the full DF object:", len(annotated_frames))
annotated_frames.head()

我们可以执行 `DataFrame.groupby().size()`，查看每 `frame_no` 的行数。

In [ ]:
# DO NOT CHANGE THIS CELL
# Check how many rows per frame_no
annotated_frames.groupby('frame_no').size()

似乎每一帧的物体数量都一样，都是 130。在此处，我们发现标记数据存在问题，那就是即便汽车已出框，但边界框仍然存在，这个问题一直持续到视频结束。我们必须使用 `outside` 列将其过滤掉。

In [ ]:
# DO NOT CHANGE THIS CELL
# Filter out annotations that do not have car inside the bbox
filtered_frames=annotated_frames[annotated_frames["outside"] == 0]
print("Length of the filtered DF object:", len(filtered_frames))
filtered_frames.head()

经过滤的 DataFrame 要小得多。我们可以绘制*包含行驶中车辆的帧索引*。

In [ ]:
# DO NOT CHANGE THIS CELL
# Plot frames that include moving cars
import matplotlib.pyplot as plt
import numpy as np

frames_list=list(filtered_frames['frame_no'].unique())
frame_existance=np.zeros(annotated_frames['frame_no'].max()+1)
for i in frames_list:
    frame_existance[int(i)]=1
y_pos=np.arange(len(frame_existance))
fig, ax=plt.subplots(figsize=(18, 3))
plt.bar(y_pos, frame_existance, align='center', alpha=0.5)
plt.title('Frame Indices that Include Moving Cars')
plt.yticks([])
plt.show()

经过滤的标记数据看起来更合理。似乎很多帧都存在没有车辆的情况。

<a name='s3.3'></a>
### 将视频文件转换为帧图像 ###
由于物体检测模型按基于帧的数据运行，因此我们需要从原始影片文件中生成帧。为此，我们将使用 [OpenCV](https://opencv.org/) 打开视频文件，并为带有标记数据的每个帧编写一个 `.png` 图像文件。我们将以 10 FPS 的速度使用原始 `.mp4` 文件。除将视频帧转换为 `.png` 图像之外，我们还将创建一个视频，将标记数据显示为边界框。

执行以下单元以创建带标记数据的视频，并为 TAO 工具包提取带标记的图像。整个流程最多需要 5 分钟。

In [ ]:
# DO NOT CHANGE THIS CELL
# Define function to extract images and generate an annotated video
import cv2
colors = [(255, 255, 0), (255, 0, 255), (0, 255, 255), (0, 0, 255), (255, 0, 0), (0, 255, 0), (0, 0, 0), (255, 100, 0), (100, 255, 0), (100, 0, 255), (255, 0, 100)]

def save_images(video_path, image_folder, frames_list, annotated_frames,  video_out_folder, fps=10):
    # Create image folder if it doesn't exist
    if not os.path.exists(image_folder):
        print("Creating images folder")
        os.makedirs(image_folder)
    
    # Create directory for output video
    if not os.path.exists(video_out_folder):
        print("Creating video out folder")
        os.makedirs(video_out_folder)
    
    # Start reading input video
    input_video=cv2.VideoCapture(video_path)
    
    # cv2.VideoCapture().read() returns true if it has a next frame
    retVal, im=input_video.read()
    size=im.shape[1], im.shape[0]
    fourcc=cv2.VideoWriter_fourcc('h','2','6','4') 
    
    # Start writing output video
    output_video=cv2.VideoWriter('{}/annotated_video.mp4'.format(video_out_folder), fourcc, fps, size)

    frameCount=0
    i=1
    
    # While has next frame
    while retVal:
        print("\rProcessing frame no: {}".format(frameCount), end='', flush=True)
        
        # If current frame is in the list of annotated frames, draw bounding box(es) and include in the output video
        if frameCount in frames_list:
            print("\rSaving frame no: {}, index: {} out of {}".format(frameCount, i, len(frames_list)), end='')
            cv2.imwrite(os.path.join(image_folder, '{}.png'.format(frameCount)), im)
            i+=1
            frame_items=annotated_frames[annotated_frames["frame_no"]==int(frameCount)]
            for index, box in frame_items.iterrows():
                xmin, ymin, xmax, ymax = box["xmin"], box["ymin"], box["xmax"], box["ymax"]
                xmin2, ymin2, xmax2, ymax2 = box["crop"][0], box["crop"][1], box["crop"][2], box["crop"][3]
                cv2.rectangle(im, (xmin, ymin), (xmax, ymax), colors[0], 1)
                cv2.rectangle(im, (int(xmin2), int(ymin2)), (int(xmax2), int(ymax2)), colors[1], 1)
            output_video.write(im)

        # Read next frame
        retVal, im=input_video.read()
        frameCount+=1

    input_video.release()
    output_video.release()
    return size        

In [ ]:
# DO NOT CHANGE THIS CELL
# Extract images and generate an annotated video
save_images('{}/126_206-A0-3_raw.mp4'.format(os.environ['SOURCE_DATA_DIR']), 
            '{}/{}'.format('{}/training'.format(os.environ['DATA_DIR']), 'images'),
            frames_list,
            filtered_frames,
            '{}/{}'.format(os.environ['DATA_DIR'], 'video_out'))

In [ ]:
# DO NOT CHANGE THIS CELL
# View the annotated output video
Video('tao_project/data/video_out/annotated_video.mp4', width=720)

<a name='s3.4'></a>
### 生成标签 ###
我们还需要为每一帧生成 KITTI 格式标签，您也可参阅 [TAO 工具包用户指南](https://docs.nvidia.com/tao/tao-toolkit/text/data_annotation_format.html#label-files)了解相关说明。KITTI 格式标签文件是简单的文本文件，每个对象包含一行。每行具有多个字段。每个物体的元素总数为 15 个，具体如下所示：<br>
`class name`, `truncation`, `occlusion`, `alpha`, `xmin`, `ymin`, `xmax`, `ymax`, `height`, `weight`, `length`, `x`, `y`, `z`, `rotation_y` <br>
目前，如要使用 TAO 工具包执行检测，只需要填充分类名称和边界框坐标字段即可。这是因为 TAO 工具包训练管道仅支持对分类和边界框坐标进行训练。可将其余字段设置为 0 以将其用作占位符。

执行以下单元以生成标签。

In [ ]:
# DO NOT CHANGE THIS CELL
# Generate labels in KITTI format
label_folder='{}/training/labels'.format(os.environ['DATA_DIR'])
if not os.path.exists(label_folder):
    print("Creating labels folder")
    os.makedirs(label_folder)
for frame in sorted(frames_list): 
    current_frame=filtered_frames[filtered_frames['frame_no']==frame]
    with open('{}/{}.txt'.format(label_folder, frame), 'w') as f: 
        for i, box in current_frame.iterrows(): 
            print('Writing for frame {}'.format(frame), end='\r')
            f.write("Car 0 0 0 {} {} {} {} 0 0 0 0 0 0 0\n".format(box['xmin'], box['ymin'], box['xmax'], box['ymax']))

In [ ]:
# DO NOT CHANAGE THIS CELL
# Preview sample KITTI format labels
!cat $DATA_DIR/training/labels/20.txt

<a name='s3.5'></a>
### 转换为 TFRecord ###
TAO 工具包可将训练数据转换为 [**TFRecord**](https://www.tensorflow.org/tutorials/load_data/tfrecord) 格式，这是一种存储二进制记录序列的简单格式。按照 TFRecord 规范，将图像帧和与该帧关联的所有标记数据编码到单行内。这非常有助于加快数据迭代。使用 KITTI 格式后，TAO 工具包可帮助我们将数据轻松转换为 TFRecord 格式。我们可以使用 `dataset_convert` 子任务完成此操作。`dataset_convert` 工具需要使用配置文件作为输入，这类文件中包含以下参数：
* `kittie_config`
    * `root_directory_path (str)`：数据集根路径。
    * `image_dir_name (str)`：包含图像的目录的相对路径。
    * `label_dir_name (str)`：包含标签的目录的相对路径。
    * `partition_mode (str)`：将数据划分为折叠时所采用的方法*（"随机划分"或"按序列划分"）*。
    * `num_partitions (int)`：拆分数据的分区数（折叠数）*（默认值为 2）*。当将分区模式设置为"随机"模式时，系统会忽略此字段，因为默认情况下系统只生成两个分区：`train` 和 `val`。
    * `image_extension (str)`：图像的扩展名*（".png"、".jpg" 或 ".jpeg"）*。
    * `val_split (float)`：要分离以进行验证的数据百分比 *(0-100)* 。
    * `num_shards (int)`：每个折叠的分片数量 _(1-20)_ 。当您拥有大量样本时，将数据集拆分成多个文件有利于操作，因为这样便于系统按以下方式处理输入：1) 并行读取以提高吞吐量，2) 能够更好地打乱数据集，以提高模型性能。当数据集很大时，这一点尤为重要。您可以参阅 [TensorFlow 的 API 文档](https://www.tensorflow.org/api_docs/python/tf/data/TFRecordDataset#raises_1)，详细了解数据分片。
* `image_directory_path (str)`：数据集根路径。

生成后，您可以在多个训练实验中使用 TFRecord。

<a name='e2'></a>
#### 练习 #2 - 数据集转换 ####
使用 `dataset_convert` 子任务来生成 TFRecord 文件。

**说明**：<br>
* 将 `<FIXME>` 更改为正确值并**保存更改**，以此修改 [TFRecord 转换规格文件](spec_files/kitti_config.txt)。
* 执行以下单元以创建 TFRecord 文件。

In [ ]:
# DO NOT CHANGE THIS CELL
# View the spec file
!cat $SPEC_FILES_DIR/kitti_config.txt

In [ ]:
# kitti_config {
#   root_directory_path: "/dli/task/tao_project/data/training"
#   image_dir_name: "images"
#   label_dir_name: "labels"
#   image_extension: ".png"
#   partition_mode: "random"
#   num_partitions: 2
#   val_split: 20
#   num_shards: 10
# }
# image_directory_path: "/dli/task/tao_project/data/training"

点击 ... 以显示**答案**。

In [ ]:
# DO NOT CHANGE THIS CELL
# View dataset_convert usage
!detectnet_v2 dataset_convert --help

使用 `dataset_convert` 子任务时，`-o` 参数表示输出文件名，`-d` 参数指向检测数据集规格文件的路径，其中包含用于导出 `.tfrecord` 文件的配置文件。

In [ ]:
# DO NOT CHANGE THIS CELL
# Create directory for TFRecords and delete existing files if they exist
!mkdir -p $DATA_DIR/tfrecords && rm -rf $DATA_DIR/tfrecords/*

!detectnet_v2 dataset_convert -d $SPEC_FILES_DIR/kitti_config.txt \
                              -o $DATA_DIR/tfrecords/kitti_trainval/kitti_trainval

查看已创建的数据分片。

In [ ]:
# DO NOT CHANGE THIS CELL
# Check the shards that have been created
!ls -rlt $DATA_DIR/tfrecords/kitti_trainval/

**您做得很好**！准备就绪后，我们开始学习[下一个 Notebook](./03_model_training_with_the_TAO_Toolkit.ipynb)。

<a href="https://www.nvidia.com/dli"> <img src="images/DLI_Header.png" alt="Header" style="width: 400px;"/> </a>